# I. Giới thiệu chung
## 1. Giới thiệu bài toán
<span style="font-size: 18px">
    Ta có thể thấy protein đóng vai trò thiết yếu trong hầu như tất các quá trình của tế bào. Thông thường, nhiều protein kết hợp với nhau tại một vị trí cụ thể để thực hiện 1 nhiệm vụ, kết quả chính xác của nhiệm vụ này phụ thuộc vào loại protein nào có mặt. Như bạn thấy, sự phân bố của protein ở các tế bào khác nhau là khác nhau, làm phát sinh sự không đồng nhất về chức năng giữa các tế bào. Việc tìm ra những điểm khác biệt như vậy và lý do tại sao có sự phân bố khác nhau của protein giữa các tế bào có cùng chức năng là điều quan trọng để hiểu được cách thức hoạt động của các tế bào, tìm ra cách các căn bệnh phát triển, sau đó tìm ra các phương pháp điều trị tốt hơn cho những bệnh đó.
</span>

## 2. Dữ liệu
<b style="font-size: 18px">Human Protein Atlas - Single Cell Classification</b>
<br>
<span style="font-size: 18px">Bộ dữ liệu có 3 loại ảnh PNG bao gồm các kích thước:</span>
    
<ul style="font-size: 18px">
    <li>1728x1728</li>
    <li>2048x2048</li>
    <li>3072x3072</li>
</ul>

<span style="font-size: 18px">
    Tất cả đều được chụp bằng kính hiển vi đồng tiêu (Confocal Microscopy)
    <br>
    Mỗi ảnh được chụp trên 4 kênh → 4 ảnh:
<br>
</span>
<ul style="font-size: 18px">
    <li><code>red:</code> microtubule channels (<a href="https://vi.wikipedia.org/wiki/Vi_ống">Vi ống</a>)</li>
    <li><code>blue:</code> nuclei channels (<a href="https://vi.wikipedia.org/wiki/Nhân_tế_bào">Nhân tế bào</a>)</li>
    <li><code>yellow:</code> Endoplasmic Reticulum (ER) channels (<a href="https://vi.wikipedia.org/wiki/Mạng_lưới_nội_chất">Lưới nội chất</a>)</li>
    <li><code>green:</code> protein of interest (<a href="https://www.google.com/search?q=protein+of+interest&oq=protein+of+interest&aqs=edge..69i57j0l3j0i22i30l3.760j0j1&sourceid=chrome&ie=UTF-8">???</a>)</li>
</ul>

## 3. Nhiệm vụ
<span style="font-size: 18px">
    Có tất cả 19 nhãn cần phân lớp, 18 nhãn cho 18 loại tế bào và 1 nhãn cho những tế bào không xác định được cụ thể. 
    
    
</span>

<pre style="font-size: 18px">
0.  Nucleoplasm  
1.  Nuclear membrane   
2.  Nucleoli   
3.  Nucleoli fibrillar center   
4.  Nuclear speckles   
5.  Nuclear bodies   
6.  Endoplasmic reticulum   
7.  Golgi apparatus   
8.  Intermediate filaments  
9.  Actin filaments  
10.  Microtubules      
11.  Mitotic spindle   
12.  Centrosome   
13.  Plasma membrane   
14.  Mitochondria   
15.  Aggresome   
16.  Cytosol   
17.  Vesicles and punctate cytosolic patterns   
18.  Negative  
</pre>

<span style="font-size: 18px">
    Tất cả các ảnh được thể hiện bằng 4 bộ lọc:
</span>
<ul style="font-size: 18px">
    <li>Bộ lọc màu xanh(<code>green</code>) được sử dụng để dự đoán và gán nhãn</li>
    <li>3 bộ lọc còn lại(<code>red, blue, yellow</code>) được sử dụng để làm tài liệu tham khảo, cho thấy sự tương quan của tế bào giữa các bộ lọc</li>
</ul>
    
### Nhiệm vụ chính
<span style="font-size: 18px">
Với mỗi ảnh chúng ta đã được cho sẵn các nhãn của các tế bào xuất hiện trong ảnh.
<br><b> Nhiệm vụ của chúng ta là segment ra các tế bào và gán nhãn cho các tế bào đó.</b>
</span>

# II. Thực hiện

## 1. Phân tích dữ liệu

In [ ]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
!pip install ipyplot -q

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import glob
import os

import os
import numpy as np
import pandas as pd

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from fastai.vision.all import *
import ipyplot
import imageio
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Khởi tạo đường dẫn đến dữ liệu và đọc dữ liệu vào biến train

In [ ]:
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))


colours = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

Tạo ra 1 từ điển để biết dữ liệu dạng số ứng với tên tế bào nào.

In [ ]:
LABELS= {
    0: "Nucleoplasm",
    1: "Nuclear membrane",
    2: "Nucleoli",
    3: "Nucleoli fibrillar center",
    4: "Nuclear speckles",
    5: "Nuclear bodies",
    6: "Endoplasmic reticulum",
    7: "Golgi apparatus",
    8: "Intermediate filaments",
    9: "Actin filaments",
    10: "Microtubules",
    11: "Mitotic spindle",
    12: "Centrosome",
    13: "Plasma membrane",
    14: "Mitochondria",
    15: "Aggresome",
    16: "Cytosol",
    17: "Vesicles and punctate cytosolic patterns",
    18: "Negative"
}

Thực hiện đưa dữ liệu đầu vào thành dạng vector one-hot để thống kê sự phân bố của các nhãn dữ liệu

Dựa trên label đã được cung cấp, tạo ra 1 vector one-hot tương ứng với 19 nhãn.

Hiển thị 5 dòng đầu dữ liệu sau khi one-hot

<img src="https://i.imgur.com/uupfzQU.png">




In [ ]:
train_csv = train.copy()
train_csv['Label'] = train_csv['Label'].apply(lambda x: list(map(int,x.split("|"))))
mlb = MultiLabelBinarizer()
train_csv[list(range(19))] = mlb.fit_transform(train_csv['Label'])
train_csv.columns = ["ID", "Label"] + list(LABELS.values())
train_csv.head(5)

Vẽ biểu đồ thể hiện sự chênh lệch giữa các nhãn trong dữ liệu.

Tính tổng theo cột của các vector one-hot đã thu được ở bước trên.

<img src="https://i.imgur.com/i5eR460.png">

Dễ thấy có sự chênh lệch lớn giữa các nhãn ở trong dữ liệu.

In [ ]:
label_count = train_csv.iloc[:, 2:].sum()
px.bar(label_count)

Thử đếm số lượng các nhãn có trong 1 ảnh để xem phân bố thế nào?

Nhận thấy các nhãn của 1 ảnh cách nhau bởi dấu | nên split ra và đếm số lượng để biết số lượng nhãn trong ảnh đó.

<img src="https://i.imgur.com/gMdGNNX.png">

Dễ thấy đa số các ảnh chỉ có 1 nhãn hoặc 2 nhãn là chính, các ảnh có số nhãn là 3, 4, 5 không đáng kể.

In [ ]:
train['num_classes'] = train['Label'].apply(lambda r: len(r.split('|')))
train['num_classes'].value_counts().plot.bar(title='Examples with multiple labels', xlabel='number of labels per example', ylabel='# train examples')
plt.show()

Xem thử các kênh màu của 3 ảnh đầu tiên trông như nào?

<img src="https://i.imgur.com/ar9djAI.png">

Trông ảnh các kênh hơi áng xanh và mỗi kênh thể hiện rõ một phần của tế bào.

In [ ]:
titles = ['microtubules', 'nuclei', 'endoplasmic reticulum', 'protein of interest']
fig, axs = plt.subplots(3, 4, figsize =(16,8))
for entry in range(3):
    for channel in range(4):
        img = plt.imread(paths[entry][channel])
        axs[entry, channel].imshow(img)        
        if entry == 0:
            axs[0, channel].set_title(titles[channel])

Vậy thì xem thử 3 ảnh đầu tiên của dữ liệu khi kết hợp các kênh màu vào trông sẽ như nào?

Đọc 4 ảnh tương ứng của mỗi ảnh theo ID ảnh cộng với đuôi _red, _green, _blue, _yellow ứng với 4 kênh màu. Sau đó sử dụng np.dstack để chồng các kênh màu lên nhau.

Đây là kết quả của 4 cách dstack r_g_b_y, r_g_b, r_y_b, b_y_g

<h4>Ảnh 1</h4>
<img src="https://i.imgur.com/yQ539pA.png"/>

<h4>Ảnh 2</h4>
<img src="https://i.imgur.com/aVYb3s7.png">

<h4>Ảnh 3</h4>
<img src="https://i.imgur.com/9WOOP6u.png">


Khi chồng cả 4 kênh màu lên thì dữ liệu khá khó nhìn vì hầu như tất cả các điểm ảnh đều chuyển thành màu trắng. 

Ở ảnh r_g_b thì cho kết quả tương đối, tuy nhiên phần hào quang xung quanh các tế bào hơi mờ nhạt.

Ở ảnh r_y_b thì cho kết quả rõ nét về tế bào, nổi bật hơn là phần bao quanh nhân tế bào.

Ở ảnh b_y_g thì cho kết quả rõ nét về tế bào, nổi bật hơn là phần nhân tế bào.

In [ ]:
for i in range(3):
    img_visual = TRAIN + '/' + train_csv['ID'].iloc[i]
    r = plt.imread(img_visual + '_red' + '.png')
    g = plt.imread(img_visual + '_green' + '.png')    
    b = plt.imread(img_visual + '_blue' + '.png')
    y = plt.imread(img_visual + '_yellow' + '.png')
    fig, ax = plt.subplots(1,4, figsize=(10,20))
    img = np.dstack((r, g, b, y))
    ax[0].set_title('r_g_b_y')
    ax[0].imshow(img)
    ax[0].axis('off')

    img = np.dstack((r, g, b))
    ax[1].set_title('r_g_b')
    ax[1].imshow(img)
    ax[1].axis('off')

    img = np.dstack((r, y, b))
    ax[2].set_title('r_y_b')
    ax[2].imshow(img)
    ax[2].axis('off')

    img = np.dstack((b, y, g))
    ax[3].set_title("b_y_g")
    ax[3].imshow(img)
    ax[3].axis('off')

    plt.show()

Thử nhìn qua xem mỗi nhãn tế bào trông sẽ thế nào?

Thực hiện đi tìm ảnh đầu tiên có 1 nhãn tương ứng với từng nhãn trong 19 nhãn tế bào đã cho và chồng 3 kênh màu red, yellow, blue lên để plot ra 1 ảnh hoàn chỉnh.

Trông nó sẽ như thế này

<div class="row">
    <h4>Nucleoplasm</h4>
    <img src="https://i.imgur.com/9xd8ZKY.png">
</div>
<div class="row">
    <h4>Nuclear membrane</h4>
    <img src="https://i.imgur.com/tPx5F3d.png">
</div>
<div class="row">
    <h4>Nucleoli</h4>
    <img src="https://i.imgur.com/9tudPrE.png">
</div>
<div class="row">
    <h4>Nucleoli fibrillar center</h4>
    <img src="https://i.imgur.com/TQ9UdzE.png">
</div>
<div class="row">
    <h4>Nuclear speckles</h4>
    <img src="https://i.imgur.com/sUhlCAp.png">
</div>
<div class="row">
    <h4>Nuclear bodies</h4>
    <img src="https://i.imgur.com/FF9TP98.png">
</div>
<div class="row">
    <h4>Endoplasmic reticulum</h4>
    <img src="https://i.imgur.com/lQCnMX9.png">
</div>
<div class="row">
    <h4>Golgi apparatus</h4>
    <img src="https://i.imgur.com/YIxOoE9.png">
</div>
<div class="row">
    <h4>Intermediate filaments</h4>
    <img src="https://i.imgur.com/3qju9Hb.png">
</div>
<div class="row">
    <h4>Actin filaments</h4>
    <img src="https://i.imgur.com/t8LM06U.png">
</div>
<div class="row">
    <h4>Microtubules</h4>
    <img src="https://i.imgur.com/cyx1zel.png">
</div>
<div class="row">
    <h4>Mitotic spindle</h4>
    <img src="https://i.imgur.com/1rGfY8b.png">
</div>
<div class="row">
    <h4>Centrosome</h4>
    <img src="https://i.imgur.com/d6uQZiW.png">
</div>
<div class="row">
    <h4>Plasma membrane</h4>
    <img src="https://i.imgur.com/JnX6iz5.png">
</div>
<div class="row">
    <h4>Mitochondria</h4>
    <img src="https://i.imgur.com/suJezhj.png">
</div>
<div class="row">
    <h4>Aggresome</h4>
    <img src="https://i.imgur.com/Ubsfqvk.png">
</div>
<div class="row">
    <h4>Cytosol</h4>
    <img src="https://i.imgur.com/faLdlaT.png">
</div>
<div class="row">
    <h4>Vesicles and punctate cytosolic patterns</h4>
    <img src="https://i.imgur.com/wjWC8F4.png">
</div>
<div class="row">
    <h4>Negative</h4>
    <img src="https://i.imgur.com/pcCDnRU.png">
</div>


In [ ]:
labels = [str(i) for i in range(19)]
class_images = []
for label in labels:
    r_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_red.png'
    y_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_yellow.png'
    b_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_blue.png'
    r = imageio.imread(TRAIN + '/' + r_img)
    y = imageio.imread(TRAIN + '/' + y_img)
    b = imageio.imread(TRAIN + '/' + b_img)
    rgb = np.dstack((r,y,b))
    class_images.append(PILImage.create(rgb))

ipyplot.plot_images(images=class_images, labels=list(LABELS.values()), max_images=19, img_width=300)

Sử dụng công cụ CellSegmentator để segment các tế bào trong 1 ảnh, tạo ra 1 Mask cho mỗi ID ảnh.

Bộ dữ liệu bảo nên sử dụng ảnh _green để huấn luyện nên segment các tế bào trong ảnh cũng sẽ segment chỉ ảnh _green.

In [ ]:
NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    padding=False,
    multi_channel_model=True,
)

image = paths[4]
arrays = image_to_arrays(image)
nuclei = arrays[1]
cell = arrays[:-1]

nuc_segmentations = segmentator.pred_nuclei([nuclei])

inter_step = [[i] for i in image[:-1]]
cell_segmentations = segmentator.pred_cells(inter_step)

Hình dáng Cell Mask của 1 ảnh trông như sau

<img src="https://i.imgur.com/pkBBJOE.png">


Trông khá chuẩn, tuy nhiên vẫn thấy 1 vài tế bào quá gần nhau thì chỉ segment thành 1 tế bào (Ví dụ ở góc dưới bên trái 2 tế bào chỉ segment thành 1 vùng)

In [ ]:
# Nuclei mask
nuclei_mask = label_nuclei(nuc_segmentations[0])
# Cell masks
cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
# Plotting

r = plt.imread(image[0])
b = plt.imread(image[1])
y = plt.imread(image[2])

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(np.dstack((r,b,y)))
ax[0].set_title('image', size=20)
ax[1].imshow(cell_mask)
ax[1].set_title('Cell Mask', size=20)
plt.show()

## 2. Huấn luyện

## Ghi chú
Do việc tạo Mask cho từng ảnh khá tốn thời gian và Kaggle chỉ cho 9 tiếng 1 phiên làm việc

Nên em sử dụng thêm bộ dữ liệu Mask (hpa-mask) do Kaggle cung cấp để không mất thời gian tạo ra các Mask cho bộ dữ liệu.

Sử dụng mô hình mask-rnn từ mmdetection để huấn luyện.


Cài đặt thêm 1 vài thứ gì đó

In [ ]:
conda install cudatoolkit=10.2

In [ ]:
!rsync -a ../input/mmdetection-v280/mmdetection ../
!pip install ../input/mmdetection-v280/src/mmdet-2.8.0/mmdet-2.8.0/
!pip install ../input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
!pip install ../input/mmdetection-v280/src/addict-2.4.0-py3-none-any.whl
!pip install ../input/mmdetection-v280/src/yapf-0.30.0-py2.py3-none-any.whl
!pip install ../input/mmdetection-v280/src/mmcv_full-1.2.6-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!pip install -r ../input/mmdetection-v280/mmdetection/requirements.txt

In [ ]:
from itertools import groupby
from pycocotools import mask as mutils
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import pickle
import cv2
from multiprocessing import Pool
import matplotlib.pyplot as plt

Chuẩn bị định nghĩa đường dẫn cho bộ Mask sẽ sử dụng và đọc dữ liệu 

In [ ]:
cell_mask_dir = '../input/hpa-mask/hpa_cell_mask'    
ROOT = '../input/hpa-single-cell-image-classification/'
MAX_THRE = 4
train_or_test = 'train'

img_dir = f'../work/mmdet_v1_train'
!mkdir -p {img_dir}
df = pd.read_csv(os.path.join(ROOT, 'train.csv'))
# df = df[:100]

Hàm thực hiện đọc ảnh gốc bằng OpenCV

In [ ]:
def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.max() > 255:
        img_max = img.max()
        img = (img/255).astype('uint8')
    return img

Coco_rle_encode dùng để encode binary masks thành rles format.

Đầu vào là binary masks 2 chiều và đầu ra là dữ liệu rles format với mục đích giảm kích thước lưu trữ xuống.

Tham khảo rles format là gì [ ở đây](https://stackoverflow.com/questions/49494337/encode-numpy-array-using-uncompressed-rle-for-coco-dataset)


In [ ]:
def coco_rle_encode(mask):
    '''
    implement storing binary masks by rles format
    input:
        mask: mask of image
    output:
        rle encode format
    '''
    rle = {'counts': [], 'size': list(mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

Hàm thực hiện biểu diễn các Mask của từng tế bào trong ảnh

Tách riêng Mask thành nhiều Mask nhỏ, mỗi Mask nhỏ ứng với 1 tế bào trong ảnh.

In [ ]:
def get_rles_from_mask(image_id):
    '''
    implement get rle of all masks in image
    input:
        image_id : id of image
    return:
        list of rle
        height of image
        width of image
    '''
    img = np.load(f'{cell_mask_dir}/{image_id}.npz')['arr_0']
    rle_list = []
    for val in np.unique(img):
        if val == 0:
            continue
        binary_mask = np.where(img == val, val, 0).astype(bool)
        counts = []
        rle = coco_rle_encode(binary_mask)
        rle_list.append(rle)
    return rle_list, img.shape[0], img.shape[1]

Hàm thực hiện convert ảnh thành coco format dataset.

Do sử dụng model mask-rnn của mmdectection yêu cầu dữ liệu được chuẩn hoá về coco format hoặc pascal format.

Ở đây, em chọn sử dụng coco format.

In [ ]:
def mk_mmdet_custom_data(image_id):
    '''
    convert data to coco format
    input:
        image_id: id of image
    return:
        annotation json files in coco format
    '''
    rles, height, width = get_rles_from_mask(image_id)
    if len(rles) == 0:
        return {
            'filename': image_id+'.jpg',
            'width': width,
            'height': height,
            'ann': {}
        }
    rles = mutils.frPyObjects(rles, height, width)
    masks = mutils.decode(rles)
    bboxes = mutils.toBbox(mutils.encode(np.asfortranarray(masks.astype(np.uint8))))
    bboxes[:, 2] += bboxes[:, 0]
    bboxes[:, 3] += bboxes[:, 1]
    return {
        'filename': image_id+'.jpg',
        'width': width,
        'height': height,
        'ann':
            {
                'bboxes': np.array(bboxes, dtype=np.float32),
                'labels': np.zeros(len(bboxes)),
                'masks': rles
            }
    }

Hàm thực hiện stack 3 kênh màu R G B lại với nhau theo ID của ảnh

In [ ]:
def load_RGB_image(image_id, train_or_test='train', image_size=None):
    '''
    load image with each channels follow by stack them
    '''
    red = read_img(image_id, "red", train_or_test, image_size)
    green = read_img(image_id, "green", train_or_test, image_size)
    blue = read_img(image_id, "blue", train_or_test, image_size)
    # using rgb only here
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

Hàm thực hiện show ảnh gốc với 3 kênh màu RGB, Mask của ảnh và (Mask + ảnh gốc)

In [ ]:
def print_masked_img(image_id, mask):
    '''
    visualize image
    input:
        image_id: id of image
        mask: mask of image with above image_id
    '''
    img = load_RGB_image(image_id, train_or_test)
    
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.6)
    plt.title('Image + Mask')
    plt.axis('off')
    plt.show()

Hàm thực hiện tạo annotation cho ảnh.

Là dữ liệu đầu vào cho quá trình huấn luyện.

In [ ]:
def mk_ann(idx):
    '''
    get the annotation json files in coco format for each image
    input:
        idx
    output:
        anno: the annotation json
        image_id : id of image
    '''
    image_id = df.iloc[idx].ID
    anno = mk_mmdet_custom_data(image_id)
    img = load_RGB_image(image_id, train_or_test)
    cv2.imwrite(f'{img_dir}/{image_id}.jpg', img)
    return anno, idx, image_id

Xem thử 3 ảnh đầu tiên với 3 kênh màu RGB, Mask của ảnh và (Mask + ảnh gốc)

Kết quả như sau

<h4>Ảnh 1<h4>
<img src="https://i.imgur.com/7QhYvR8.png">
    
<h4>Ảnh 2</h4>
<img src="https://i.imgur.com/NFOUVqQ.png">
    
        
<h4>Ảnh 3</h4>
<img src="https://i.imgur.com/WM84d1k.png">
    
Mask segment khá chuẩn các tế bào trong ảnh.

In [ ]:
cell_mask_dir = '../input/hpa-mask/hpa_cell_mask'    
for idx in range(3):
    image_id = df.iloc[idx].ID
    cell_mask = np.load(f'{cell_mask_dir}/{image_id}.npz')['arr_0']
    print_masked_img(image_id, cell_mask)
    load_RGB_image(image_id)

Do bộ dữ liệu quá lớn và độ phức tập của bài toán cao. 

Nhận thấy việc phân lớp nhãn trong 1 ảnh có nhiều nhãn quá sức và cũng không đủ tài nguyên để huấn luyện nên em chỉ huấn luyện các ảnh chỉ có 1 nhãn. Cũng không có quá nhiều dữ liệu với 3,4,5 nhãn, đa phần là 1,2 nhãn chiếm phần lớn trong bộ dữ liệu.

Các ảnh chỉ có 1 nhãn là các ảnh không có ký tự | trong Label nên em thực hiện lấy ra các dòng dữ liệu chỉ có 1 nhãn.

Em tìm được <b>10412</b> ảnh chỉ có 1 nhãn trong bộ dữ liệu

<b>Một ảnh tương ứng với 4 kênh màu, tức là 4 ảnh khác  </b>

<b>10412 ảnh là 41648 ảnh</b>

In [ ]:
num_sample = df.ID.iloc[[idxx for idxx in range(len(df)) if '|' not in df['Label'].iloc[idxx]]]
len(list(num_sample))

Xem lại dữ liệu sau khi lấy ra các ảnh chỉ có 1 nhãn.

Cột đầu tiên là index của các ảnh chỉ có 1 nhãn trong bộ dữ liệu gốc phục vụ cho quá trình huấn luyện phía dưới.

<img src="https://i.imgur.com/k2ZG2yT.png">

In [ ]:
num_sample.to_frame()

Do giới hạn của Kaggle là 9 tiếng, việc huấn luyện với <b>10412</b> ảnh trong 9 tiếng liên tục bị crash mà không thu được kết quả

Nên em chỉ lấy ra 1000 ảnh (tức là 4000 ảnh) dùng để huấn luyện tự ước tính đủ trong 9 tiếng của 1 phiên làm việc.

Đọc dữ liệu và tạo annotation cho từng ảnh để làm đầu vào của quá trình huấn luyện.

In [ ]:
p = Pool(processes=MAX_THRE)
annos = []
c = 0
for i, (anno, idx, image_id) in enumerate(p.imap(mk_ann, range(len(df)))):
    if len(anno['ann']) > 0 and image_id in list(num_sample):
        annos.append(anno)
        c += 1
    if c % 100 == 0:
        print (idx, image_id)
    if c > 1000:
        break

Chia bộ dữ liệu huấn luyện thành 2 tập train và validation theo tỉ lệ <b>(train : val) : (80 : 20)</b>

In [ ]:
lbl_cnt_dict = df.set_index('ID').to_dict()['Label']
trn_annos = []
val_annos = []
val_len = int(len(annos)*20/100)
for idx in range(len(annos)):
    ann = annos[idx]
    filename = ann['filename'].replace('.jpg','').replace('.png','')
    label_id = lbl_cnt_dict[filename]
    if '|' not in label_id:
        label_id = int(label_id)
        ann['ann']['labels'] = np.full(len(ann['ann']['bboxes']), label_id)
        if idx < val_len:
            val_annos.append(ann)
        else:
            trn_annos.append(ann)

In [ ]:
print (len(trn_annos))
print (len(val_annos))

In [ ]:
with open(f'../work/mmdet_v1_full.pkl', 'wb') as f:
    pickle.dump(annos, f)
with open(f'../work/mmdet_v1_trn.pkl', 'wb') as f:
    pickle.dump(trn_annos, f)
with open(f'../work/mmdet_v1_val.pkl', 'wb') as f:
    pickle.dump(val_annos, f)

Do cấu hình mặc định của mô hình mask-rnn huấn luyện không sử dụng tập validation

Nên em phải viết lại config để mô hình có thể huấn luyện sử dụng tập validation để tự đánh giá lại mô hình.

Quá trình train được để bên dưới

Do Kaggle chỉ có 9 tiếng cho 1 phiên làm việc làm việc train hay bị crash nên em có chụp lại kết quả quá trình train, chỉ chạy được đến epoch thứ 10 là bị crash do hết phiên làm việc.

Em phải copy lại kết quả trong quá trình train để vẽ lại Train/Val loss



<img src="https://i.imgur.com/FAqWUQb.png">

<img src="https://i.imgur.com/E6UwG8D.png">

In [ ]:
# config = f'configs/hpa_{exp_name}/mask_rcnn_r50_fpn_1x_coco.py'
config = f'configs/mask_rcnn_unique/mask_rcnn_r50_fpn_1x_coco.py'

# using --no-validate to avoid some errors for custom dataset metrics
# additional_conf = '--no-validate '
additional_conf = ' --cfg-options workflow="[(train,1),(val,1)]"'
additional_conf += f' --cfg-options optimizer.lr=0.0025'
additional_conf += f' --cfg-options work_dir=../working/work_dir'
additional_conf += f' --cfg-options load_from=../input/mmdetection-v280/pretrained/mask_rcnn_r50_fpn_2x_coco_bbox_mAP-0.392__segm_mAP-0.354_20200505_003907-3e542a40.pth'
cmd = f'bash -x tools/dist_train.sh {config} 1 {additional_conf}'
!cd ../mmdetection; {cmd}

## 3. Test

Định nghĩa lại một vài biến để phục vụ quá trình test.

Đọc dữ liệu test dựa trên file sample_submission mà Kaggle cung cấp.

In [ ]:
from pycocotools import _mask as coco_mask
import matplotlib.pyplot as plt
import os
import base64
import typing as t
import zlib
import random
random.seed(0)

exp_name = "mmdet_v1"
image_size = None
ROOT = '../input/hpa-single-cell-image-classification/'
train_or_test = 'test'
df = pd.read_csv(os.path.join(ROOT, 'sample_submission.csv'))

Tạo annotation cho dữ liệu test vì đầu vào của mô hình sử dụng annotation dưới dạng coco format

In [ ]:
out_image_dir = f'../work/mmdet_v1_test/'
!mkdir -p {out_image_dir}

annos = []
for idx in tqdm(range(len(df))):
    image_id = df.iloc[idx].ID
    img = load_RGB_image(image_id, train_or_test, image_size)
    
    cv2.imwrite(f'{out_image_dir}/{image_id}.jpg', img)
    ann = {
        'filename': image_id+'.jpg',
        'width': img.shape[1],
        'height': img.shape[0],
        'ann': {
            'bboxes': None,
            'labels': None,
            'masks': None
        }
    }
    annos.append(ann)
    
with open(f'../work/mmdet_v1_tst.pkl', 'wb') as f:
    pickle.dump(annos, f)

Thực hiện quá trình gán nhãn cho các ảnh trong tập test

In [ ]:
config = 'configs/mask_rcnn_unique/mask_rcnn_r50_fpn_1x_coco.py'
model_file = './epoch_10.pth'
result_pkl = '../work/mask_rcnn_r50_fpn_1x_epoch_10.pkl'
cmd = f'python tools/test.py {config} {model_file} --out {result_pkl}'
!cd ../mmdetection; {cmd}
result = pickle.load(open('../mmdetection/'+result_pkl, 'rb'))

Xem thử kết quả của 3 ảnh đầu tiên trong tập test.

Kết quả cho thấy 
* class_id: là nhãn được gán nhãn ứng với ảnh đó
* image_id: là ID của ảnh trong tập test
* confidence: là độ tin cậy mà mô hình gán nhãn cho tế bào trong ảnh


<img src="https://i.imgur.com/O13HpT3.jpg">

In [ ]:
for ii in range(3):
    image_id = annos[ii]['filename'].replace('.jpg','').replace('.png','')
    for class_id in range(19):
        bbs = result[ii][0][class_id]
        sgs = result[ii][1][class_id]
        for bb, sg in zip(bbs,sgs):
            box = bb[:4]
            cnf = bb[4]
            h = sg['size'][0]
            w = sg['size'][0]
            if cnf > 0.3:
                print(f'class_id:{class_id}, image_id:{image_id}, confidence:{cnf}')
                mask = mutils.decode(sg).astype(bool)
                print_masked_img(image_id, mask)